In [3]:
import pandas as pd
from google.colab import drive
import re
import torch
import warnings
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

In [4]:
drive.mount('/content/drive') # mount

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Augnito/mtsamples.csv') # read the csv file
print(df.head())  #print the dataset


   Unnamed: 0                                        description  \
0           0   A 23-year-old white female presents with comp...   
1           1           Consult for laparoscopic gastric bypass.   
2           2           Consult for laparoscopic gastric bypass.   
3           3                             2-D M-Mode. Doppler.     
4           4                                 2-D Echocardiogram   

             medical_specialty                                sample_name  \
0         Allergy / Immunology                         Allergic Rhinitis    
1                   Bariatrics   Laparoscopic Gastric Bypass Consult - 2    
2                   Bariatrics   Laparoscopic Gastric Bypass Consult - 1    
3   Cardiovascular / Pulmonary                    2-D Echocardiogram - 1    
4   Cardiovascular / Pulmonary                    2-D Echocardiogram - 2    

                                       transcription  \
0  SUBJECTIVE:,  This 23-year-old white female pr...   
1  PAST MEDICAL 

In [6]:
print(df.shape) # print shape

(4999, 6)


In [7]:
df = df.loc[:, ["transcription", "keywords"]] # Filter the columns
df.head()

,transcription,keywords
0,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [8]:
df= df.dropna(subset=['transcription', 'keywords']) # remove null values
df.shape

(3898, 2)

In [9]:
df = df.drop_duplicates(subset=['transcription', 'keywords']) # remove duplicates rows
df.shape


(3852, 2)

In [10]:
def clean_text(text):
    return re.sub(r'[^a-z0-9\s]', '', text.lower()) # remove special character and convert into lowercase

In [11]:
df[['transcription', 'keywords']] = df[['transcription', 'keywords']].applymap(clean_text)
print(df.head()) # print cleaned data

                                       transcription  \
0  subjective  this 23yearold white female presen...   
1  past medical history he has difficulty climbin...   
2  history of present illness  i have seen abc to...   
3  2d mmode  1  left atrial enlargement with left...   
4  1  the left ventricular cavity size and wall t...   

                                            keywords  
0  allergy  immunology allergic rhinitis allergie...  
1  bariatrics laparoscopic gastric bypass weight ...  
2  bariatrics laparoscopic gastric bypass heart a...  
3  cardiovascular  pulmonary 2d mmode doppler aor...  
4  cardiovascular  pulmonary 2d doppler echocardi...  


In [12]:
!pip install sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f5577c529ffc59de6c4c4fc67c8cbdeceb5644627cb3ab53ec3e894c1cf27430
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [13]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
import torch

In [14]:
model = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
if torch.cuda.is_available():
  model = model.to('cuda')

In [16]:
class CustomDataset(Dataset):
    def __init__(self, df, transcript, keywords, tokenizer):
        self.df = df
        self.transcript = transcript
        self.keywords = keywords
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        transcript_text = self.df.loc[idx, self.transcript]
        keywords_text = self.df.loc[idx, self.keywords]

        encoding_transcript = self.tokenizer(transcript_text, padding='max_length', truncation=True, return_tensors='pt')
        encoding_keywords = self.tokenizer(keywords_text, padding='max_length', truncation=True, return_tensors='pt')

        transcript_tokens = encoding_transcript['input_ids'][0]
        keywords_tokens = encoding_keywords['input_ids'][0]

        if torch.cuda.is_available():
            transcript_tokens = transcript_tokens.to('cuda')
            keywords_tokens = keywords_tokens.to('cuda')

        return transcript_tokens, keywords_tokens


In [17]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=0.9, random_state=32)
train_data = CustomDataset(df_train, 'transcription', 'keywords', tokenizer)
test_data = CustomDataset(df_test, 'transcription', 'keywords', tokenizer)
batch_size = 5

In [18]:
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)